# Parsing of Sustainability Reports in PDF format

## Setup

In [1]:
import os
import json
import re


REPORT_DIRECTORY = './00_data/reports'
REPORTS = os.listdir(REPORT_DIRECTORY)
PARSING_RESULTS_DIRECTORY = '00_data/parsing_results'
IMAGES = '00_data/parsing_results/pdf_images/'

## Parsing with pdfminer.six

In [2]:
from pdfminer.high_level import extract_text

def cleanse(text):
    return re.sub(' +', ' ', text.strip())
    
def parse_pdf_pdfminer(file):
    # Read pdf using pdfminer
    text = extract_text(file)

    # Paragraphs are delimited by double newlines
    paragraphs = {}
    paragraph_index = 0

    # Add each paragraph with its index to a dict
    for paragraph in text.split('\n\n'):
        paragraph = cleanse(paragraph)
        sentences = paragraph.split('.')
        if len(sentences) > 0:
            sentence_index = 0
            for sentence in paragraph.split('.'):
                sentence = sentence + '.'
                if paragraph_index not in paragraphs:
                    paragraphs[paragraph_index] = {}
                
                paragraphs[paragraph_index][sentence_index] = sentence
                sentence_index += 1

            paragraph_index += 1

    return paragraphs



In [3]:

for file in REPORTS:
    print('{}/{}'.format(REPORT_DIRECTORY, file))
    parsed_text = parse_pdf_pdfminer('{}/{}'.format(REPORT_DIRECTORY, file), )

    with open('{}/{}_{}.json'.format(PARSING_RESULTS_DIRECTORY, 'pdfminer', file[:-4]), 'w') as f:
        json.dump(parsed_text, f, indent=4)   

./00_data/reports/Arbonia_Sustainability_Report_2021.pdf
./00_data/reports/Firmenich_KOZQjRz6ornoYorSa601gLSGvA830xU4JD2UzZghMew.pdf
./00_data/reports/Nestle_creating-shared-value-sustainability-report-2021-en.pdf


## Parsing with OCR

In [4]:
from pdf2jpg import pdf2jpg
import easyocr

def cleanse(text):
    return re.sub(' +', ' ', text)

reader = easyocr.Reader(['en'])

for file in REPORTS:
    print('{}/{}'.format(REPORT_DIRECTORY, file))

    result = pdf2jpg.convert_pdf2jpg('{}/{}'.format(REPORT_DIRECTORY, file), IMAGES, dpi=300, pages='ALL')

    paragraphs = {}
    paragraph_index = 0

    for img in os.listdir('{}/{}_dir'.format(IMAGES, file)):
        text = reader.readtext('{}/{}_dir/{}'.format(IMAGES, file, img), paragraph=True)
        for paragraph in text:
            paragraph = cleanse(paragraph[1])
            sentence_index = 0
            for sentence in paragraph.split('.'):
                sentence = sentence + '.'
                if paragraph_index not in paragraphs:
                    paragraphs[paragraph_index] = {}
                
                paragraphs[paragraph_index][sentence_index] = sentence
                sentence_index += 1

            paragraph_index += 1

    with open('{}/{}_{}.json'.format(PARSING_RESULTS_DIRECTORY, 'easyocr', file[:-4]), 'w') as f:
        json.dump(paragraphs, f, indent=4)
    

ModuleNotFoundError: No module named 'pdf2jpg'